In [1]:
import numpy as np
import pandas as pd
from sas7bdat import SAS7BDAT

In [2]:
file_path = 'individus_ct2013.sas7bdat'
with SAS7BDAT(file_path) as reader:
    df = reader.to_data_frame()
df.info()

[individus_ct2013.sas7bdat] column count mismatch


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33673 entries, 0 to 33672
Columns: 541 entries, C1C to revmenscc_drap
dtypes: float64(76), object(465)
memory usage: 139.0+ MB


# Créer une variable binaire pour indiquer si la femme est le principal apporteur de ressources

In [3]:
df = df.query('COUPLE == "1"')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24024 entries, 2 to 33672
Columns: 541 entries, C1C to revmenscc_drap
dtypes: float64(76), object(465)
memory usage: 99.3+ MB


In [4]:

# Créer la nouvelle colonne 'femme_principal_apporteur' avec des valeurs 1 ou 0
df['femme_principal_apporteur'] = df.apply(lambda row: 1 if (row['PRACT'] == '1' and row['SEXE'] == '2') else 0, axis=1)

# Compter le nombre de lignes où 'femme_principal_apporteur' est égal à 1
nombre_de_femmes_principal_apporteur = df['femme_principal_apporteur'].sum()

print("Nombre de lignes où 'femme_principal_apporteur' est égal à", nombre_de_femmes_principal_apporteur)


Nombre de lignes où 'femme_principal_apporteur' est égal à 5080


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24024 entries, 2 to 33672
Columns: 542 entries, C1C to femme_principal_apporteur
dtypes: float64(76), int64(1), object(465)
memory usage: 99.5+ MB
